In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import time
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
from scipy.interpolate import RectBivariateSpline
from module.interpolate import Interpolator

---

# Problem 1


## ( 1-a )
- We will first consider a single particle starting from an initial position $x_0 = [1.05,0.5]$. 
- Let the trajectory, x(t), of the particle be controlled by Eqs. (3) and (4). Calculate the trajectory for the time interval $ t_1 \in [0, 50]$,  using Heun’s method.
- Try a few different timesteps, and compare the results by plotting the trajectories for different timesteps in the same plot. Make up your mind about what seems a reasonably short timestep. 
- What happens if you double the integration time, such that $ t_2 \in [0, 100]$, is the same timestep still a good choice?



In [ ]:
fig_params = {"figsize": (7.5, 5)}
streamplot_params = {
    "color": "seagreen",
    "density": 1,
    "linewidth": 0.25,
    "arrowsize": 1,
    "arrowstyle": "->",
}
scatter_params = {"s": 5}
plot_params = {"alpha":1, "linewidth":0.8}

In [ ]:
from module.utilities import run_steplength_test


def velocity(X, t):
    A, eps, w = 0.10, 0.25, 1  # Initial constants
    a = eps * np.sin(w * t)  # Equation 4a
    b = 1 - 2 * eps * np.sin(w * t)
    x, y = X  # Assuming X is a 2-element array or has shape (2, N)
    f = a * x**2 + b * x  # Equation 4b
    dx = -np.pi * A * np.sin(np.pi * f) * np.cos(np.pi * y)  # Equation 3
    dy = np.pi * A * np.cos(np.pi * f) * np.sin(np.pi * y) * (2 * a * x + b)
    return np.array([dx, dy])


x0 = np.array([[1.05], [0.50]])
T_values = np.array([50, 100])
dt_values = np.array([1, 0.1, 0.01, 0.001])

fig, ax = plt.subplots(1, len(T_values), figsize=(12, 5))
ax, time_array = run_steplength_test(
    velocity, ax, x0=x0, T_values=T_values, dt_values=dt_values, **plot_params
)
fig.suptitle("Trajectory for different step lengths")
fig.supxlabel("X")
fig.supylabel("Y")
fig.legend(*ax[0].get_legend_handles_labels(), loc="upper right", title="Step length")
ax[0].grid()
ax[1].grid()
plt.tight_layout()
plt.show()


fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(dt_values, time_array[0, :], label="T = 50")
ax.plot(dt_values, time_array[1, :], label="T = 100")

ax.set_xscale("log")
ax.set_title("Runtime vs. timestep")
ax.set_xlabel("timestep")
ax.set_ylabel("runtime")
ax.grid()
ax.legend()
plt.show()

In [ ]:
import time
from module.trajectory import Trajectory

x0 = np.array([[1.05], [0.50]])
T_values = np.array([50, 100])
dt_values = np.array([1, 0.1, 0.01, 0.001])


figd, axd = plt.subplot_mosaic(
    [["top", "top"], ["left", "right"]], constrained_layout=True
)

x0 = np.array([[1.05], [0.50]])


for tf in T_values:
    t_array = np.empty((len(dt_values)))
    for dt in dt_values:
        print(f'T:{tf:^10} dt:{dt:^10}')
        test_traj = Trajectory(x0,(0,tf),dt,Np=100)
        tik = time.time()
        test_traj(velocity, 'heun')
        tok = time.time()
        np.append(t_array, tik-tok)
    print(t_array.shape)
    axd['top'].plot(dt_values, t_array)
    axd['top'].set_xscale("log")
    axd['top'].set_title("Runtime vs. timestep")
    axd['top'].set_xlabel("timestep")
    axd['top'].set_ylabel("runtime")
plt.show()

For the first time interval $t \in [0, 50]$, the time-step $t = 0.1$ seems to be suffiecient to represent the particle.
For the time interval $t \in [0, 100]$, $t = 0.1$ deviates more from the shorter time steps, and is not short enough to represent the particle path correctly. With $t = 0.01$ however, is a better approximation, when comparing to the smaller timestep. 

Regarding the runtime, the biggest reduction happens when increasing $dt$ from 0.001 to 0.01. The graph shows a double in runtime when the time interval doubles, i.e. runtime is proportional to array length when using numpy arrays. For the time intervals tested here, the length of the arrays is not long enough for $dt$ to have a remarkable impact on runtime. For calculations going over several days, which will be the case later in the rapport, $dt$ will have to be significantly bigger.

Due to the significant reduction in runtime, $t = 0.01$ is the most economic timestep for these calculations.


## ( 1-b )

In [ ]:
fig_params = {"figsize": (7.5, 5)}
streamplot_params = {
    "color": "grey",
    "density": 1,
    "linewidth": 0.25,
    "arrowsize": 1,
    "arrowstyle": "->",
}
scatter_params = {"s": 5}
plot_params = {"alpha":1, "linewidth":0.8}


T = [0, 10]
dt = 0.001
Np = 100
mode = "grid"
loc = [-0.1, 0.1]
scale = 0.1
fig, ax = plt.subplots(1,1, figsize=(10, 10))
traj = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale)
Y = traj(velocity)
X, Y = np.meshgrid(np.linspace(-1.5,1.3, 100), np.linspace(-0.99, 0.99, 100))
U, V = velocity([X, Y], 0)
ax.streamplot(X, Y, U, V, **{"color": "gray", "density": 1, "linewidth": 0.5, "arrowsize": 0.9, "arrowstyle": "->"})
traj.plot(ax, **plot_params)
traj.scatter(ax, t=0, **scatter_params)
traj.scatter(ax, t=-1, **scatter_params)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title(f"Trajectories of $N_p = 100$ particles in $(0.1\\times 0.1)$ grid")
plt.grid()
plt.tight_layout()
plt.show()

Depending on the initial position of each particle, here only separated by a distance of 0.01 units, takes different paths. The time interval is here limited to $t \in [0, 10]$ The streamlines represent the velocity field at $t=0$ to give an idea of the impact of the current and wind. This will change with time, but keeps somewhat the same shape. The particles on the further right are caught by the streams to the right, and similarly on the left side. 

## ( 1-c )


In [ ]:
from  module.utilities import run_timing_test

dt = 0.01
T = [0, 10]
Np_values = [1, 10, 100, 1000, 10_000,100_000]
times, linear_times, = run_timing_test(velocity, T=T, Np_values=Np_values, dt=0.01)

fig, ax = plt.subplots(figsize=(7.5, 5))
ax.plot(Np_values, times, marker="o", label="Actual Time")
ax.plot(Np_values, linear_times, marker="o", linestyle="--", label="Linear Time")
ax.set_title("Run Time vs. Number of Particles")
ax.set_xlabel("Number of Particles")
ax.set_ylabel("Run Time (s)")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()
plt.grid(True)
plt.show();


The graph shows the curve is approximately constant for fewer particles, i.e. shorter arrays. For few particles, the linear increase may be too little to measure. The overhead time, in this case, is the time that python and jupyter cell/kernel uses as preparation for executing the task at hand. This overhead time can be few milliseconds, meaning if the running time of the numerical calculations are much smaller than the overhead time, there are no noticable variations between runtimes before it reaches a certain threshold. Here, this threshold seems to be reached when there are over 100 particles.

Another possible explanation for the constant runtime is how numpy allocates memory. For smaller array-sizes the compiler may overestimate how much memory needs to be allocated to avoid asking for new memory for every inceased index. Adding elements threrfore doesn't require additional runtime, resulting in constant time $\mathcal{O}(1)$. For bigger array-sizes, Python needs to allocate memory more frequently, resulting in a linear increase of runtime $\mathcal{O}(n)$.


---


# Problem 2


## ( 2-a )

$\hat{x} = 790.000$ og $\hat{y} = 490.000$


In [ ]:
streamplot_params = {
    "color": "seagreen",
    "density": 1,
    "linewidth": 0.25,
    "arrowsize": 1,
    "arrowstyle": "->",
}
scatterInitParams = {"s": 5, "color": "blue"}
scatterFinalParams = {"s": 5, "color": "red"}
plot_params = {"alpha": 0.5, "linewidth": 0.25,"color":"cornflowerblue"}

In [ ]:
d = xr.open_dataset("data/NorKyst-800m.nc")
f = Interpolator(dataset=d)

Np = 500
days, dt = 5, 3600
T = [0, days * 24 * dt]
mode = "map"
loc, scale = [790000, 490000], 1000

fig, ax = plt.subplots(1,1,figsize=(12, 8), subplot_kw={"projection": ccrs.PlateCarree()})
traj.init_map(ax, **plot_params)
traj.scatter(ax, t=0, label="initial position", **scatterInitParams)
traj.scatter(ax, t=-1, label="final position", **scatterFinalParams)
ax.set_title(f"Trajectories of $N_p = {Np}$ particles over {days} days")
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")

traj = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale)
traj(f)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
traj.plot(ax, **plot_params)
traj.scatter(ax, t=0, label="initial position", **scatterInitParams)
traj.scatter(ax, t=-1, label="final position", **scatterFinalParams)
ax.set_title(f"Trajectories of $N_p = {Np}$ particles over {days} days")
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
plt.tight_layout()
plt.show()

## ( 2-b )

In [ ]:
streamplot_params = {
    "color": "seagreen",
    "density": 1,
    "linewidth": 0.25,
    "arrowsize": 1,
    "arrowstyle": "->",
}
scatterInitParams = {"s": 5, "color": "blue"}
scatterFinalParams = {"s": 5, "color": "green"}
plot_params = {"alpha": 0.5, "linewidth": 0.20,"color":"cornflowerblue"}

In [ ]:
d = xr.open_dataset("data/NorKyst-800m.nc")
f = Interpolator(dataset=d)

Np=500
days = 5
T = [0, days * 24 * 3600]
dt=3600
mode = "map"

fig, ax = plt.subplots(1,3,figsize=(12, 12), subplot_kw={"projection": ccrs.PlateCarree()})

ax[0].set_title("Bergensfjorden")
loc, scale = [250000, 460000], 10000
traj_bergen = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
traj_bergen(f)
traj_bergen.init_map(ax[0], **plot_params)
traj_bergen.streamplot(ax[0], **streamplot_params)
traj_bergen.scatter(ax[0], t=0, **scatterInitParams)
traj_bergen.scatter(ax[0], t=-1, **scatterFinalParams)

ax[1].set_title("Lofoten")
loc, scale = [1240000, 480000], 10000
traj_lofoten = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
traj_lofoten(f)
traj_lofoten.init_map(ax[1], **plot_params)
traj_lofoten.streamplot(ax[1], **streamplot_params)
traj_lofoten.scatter(ax[1], t=0, **scatterInitParams)
traj_lofoten.scatter(ax[1], t=-1, **scatterFinalParams)

ax[2].set_title("Oslofjorden")
loc, scale = [200_000, 140_000], 1000
traj_oslo = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
traj_oslo(f)
traj_oslo.init_map(ax[2], **plot_params)
traj_oslo.streamplot(ax[2], **streamplot_params)
traj_oslo.scatter(ax[2], t=0, **scatterInitParams)
traj_oslo.scatter(ax[2], t=-1, **scatterFinalParams)
plt.show()

# Problem 3

## ( 3-a )


In [ ]:
streamplot_params = {
    "color": "seagreen",
    "density": 0.5,
    "linewidth": 1,
    "arrowsize": 1,
    "arrowstyle": "->",
}
plot_params = {
    "color": "cornflowerblue",
    "alpha" : 0.5,
    "linewidth": 0.1
}

In [ ]:
d = xr.open_dataset("data/NorKyst-800m.nc")
f = Interpolator(dataset=d)

Np=1000
days = 5
T = [0, days * 24 * 3600]
dt=3600
mode = "map"
loc, scale = [250000, 460000], 10000

fig, ax = plt.subplots(1,3,figsize=(12, 12), subplot_kw={"projection": ccrs.PlateCarree()})
traj_bergen = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
traj_bergen(f)
traj_bergen.init_map(ax[0], **plot_params)
traj_bergen.streamplot(ax[0], **streamplot_params)
traj_bergen.scatter(ax[0], t=0, **{"label": "Initial Position", "color": "blue", "s": 5})
traj_bergen.scatter(ax[0], t=-1, **{"label": "Final Position", "color": "green", "s": 5})
ax[0].set_title("Bergensfjorden")

loc, scale = [1240000, 480000], 10000
traj_lofoten = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
traj_lofoten(f)
traj_lofoten.init_map(ax[1], **plot_params)
traj_lofoten.streamplot(ax[1], **streamplot_params)
traj_lofoten.scatter(ax[1], t=0, **{"label": "Initial Position", "color": "blue", "s": 5})
traj_lofoten.scatter(ax[1], t=-1, **{"label": "Final Position", "color": "green", "s": 5})
ax[1].set_title("Lofoten")

loc, scale = [35000, 14000], 1000
traj_oslo = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
traj_oslo(f)
traj_oslo.init_map(ax[2], **plot_params)
traj_oslo.streamplot(ax[2], **streamplot_params)
traj_oslo.scatter(ax[2], t=0, **{"label": "Initial Position", "color": "blue", "s": 5})
traj_oslo.scatter(ax[2], t=-1, **{"label": "Final Position", "color": "green", "s": 5})
ax[2].set_title("Oslofjorden")
plt.tight_layout()
plt.show()

## ( 3-b ) Windage factor

With the same initial conditions as above, try a few different windage factors, in
the range from 0 to 0.15, and show how this affects the results. You can for example
look into the percentage of particles that become stranded after three days as a
function of fw. Note that the results may of course depend on where (and when)
you start the particle trajectories, and for some initial conditions no particles will
reach land at all


In [ ]:
fw = np.linspace(0, 0.30, 10)

Np=100
days = 3
T = [0, days * 24 * 3600]
dt=3600
mode = "map"
loc, scale = [250000, 460000], 10000
fig, ax = plt.subplots(1,1,figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})

landed = []
for w in fw:
    d = xr.open_dataset("data/NorKyst-800m.nc")
    f = Interpolator(dataset=d)
    traj_bergen = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
    traj_bergen(f)
    traj_bergen.init_map(ax, **plot_params)
    traj_bergen.plot(ax, **{"alpha":0.5, "lw":0.25})
    traj_bergen.scatter(ax, t=0, **{"label": "Initial Position","s":2.5})
    traj_bergen.scatter(ax, t=-1, **{"label": "Final Position", "s":2.5})
    landed.append(traj_bergen.get_land_particles())


ax.set_title("Bergenskysten")
plt.show()

In [ ]:
fw = np.linspace(0, 0.30, 10)
streamplot_params = {"density":0.5, "color":"seagreen",}
Np=100
days = 3
T = [0, days * 24 * 3600]
dt=3600
mode = "map"
loc, scale = [1240000, 480000], 10000
fig, ax = plt.subplots(1,1,figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})

landed = []
for w in fw:
    d = xr.open_dataset("data/NorKyst-800m.nc")
    f = Interpolator(dataset=d)
    traj_bergen = Trajectory(T=T, dt=dt, Np=Np, mode=mode, loc=loc, scale=scale, check_land=True)
    traj_bergen(f)
    traj_bergen.init_map(ax, **plot_params)
    traj_bergen.plot(ax, **{"alpha":0.5, "lw":0.25})
    traj_bergen.scatter(ax, t=0, **{"label": "Initial Position","s":2.5})
    traj_bergen.scatter(ax, t=-1, **{"label": "Final Position", "s":2.5})
    traj_bergen.streamplot(ax, **streamplot_params)
    landed.append(traj_bergen.get_land_particles())


ax.set_title("Bergenskysten")
plt.show()

In [ ]:

def get_landed_total(landed):
    coast = []
    for x,y in landed:
        coast.append(x.shape[0])
    coast = np.array(coast)
    return coast

coast = get_landed_total(landed)

fig, ax = plt.subplots(1,1,figsize=(10, 5))
ax.plot(fw, coast, marker="o")
ax.set_xlabel("Windage factor")
ax.set_ylabel("Percentage of particles landed")
ax.set_title("Percentage of particles landed vs. windage factor")
plt.grid()
plt.show()